In [13]:
from google.colab import drive
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.metrics import Accuracy
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import model_selection
#https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#DATA PREPARATION
raw_datas = pd.read_csv("drive/My Drive/Dann Dataset/PIR dataset Time train.csv")
number_of_day = raw_datas.shape[0]/86400
print("Total data = " + str(raw_datas.shape[0]) + " detik")
print("Total hari dalam data = " + str(int(number_of_day)) + " hari")

Total data = 604800 detik
Total hari dalam data = 7 hari


In [ ]:
#EXTRACTING DATA

datas = pd.read_csv("drive/My Drive/Dann Dataset/PIR dataset Duration train.csv")
file = open("/content/drive/MyDrive/Dann Dataset/PIR last 30 list dataset train.csv","w")
for z in range(30):
  file.write("Durasi")
  file.write(str(z+1))
  for y in range(4):
    file.write(",Data")
    file.write(str(z+1))
    file.write("|")
    file.write("Pir")
    file.write(str(y+1))
  file.write(",")
file.write("Gerakan")
file.write("\n")

percent1 = round(datas.shape[0]/100)
percentage = 0

for i in range(datas.shape[0]):
  if(i%percent1 == 0):
    print(str(percentage)+"%")
    percentage+=1
  if(i < 29):
    for u in range(29-i):
      file.write("0,0,0,0,0,")
    for e in range(i+1):
      for t in range(5):
        file.write(str(datas.iloc[e][t]))
        file.write(",") 
  else:
    for a in range(30):
      for o in range(5):
        file.write(str(datas.iloc[i-(29-a)][o]))
        file.write(",")
  file.write(datas.iloc[i][5])
  file.write("\n")

print("100%")
file.close()

0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%
100%


In [15]:
# LOAD DATASET
feature_data =pd.read_csv("drive/MyDrive/Dann Dataset/PIR last 30 list dataset train.csv")

# ADD LABEL ENCODER
label_encoder = LabelEncoder()
feature_data['Gerakan'] = label_encoder.fit_transform(feature_data['Gerakan'])
datas_labels = label_encoder.classes_

In [16]:
# LIST OF LABELS
print(datas_labels)

['LOITERING' 'NORMAL']


In [17]:
x = feature_data.drop('Gerakan',axis=1)
y = feature_data['Gerakan']

# RESHAPE DATASET
x = x.values.reshape(-1,30,5)   #INPUT 4
y = y.values.reshape((-1,1))

# SPLIT DATASET
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3) #Split Dataset Train 70% dan Test 30%

# CHECK THE SHAPE
print("Train:", x_train.shape , y_train.shape)
print("Test:",x_test.shape , y_test.shape)

# SET TOTAL FOLD
total_fold = 5

Train: (140202, 30, 5) (140202, 1)
Test: (60087, 30, 5) (60087, 1)


In [ ]:
#RNN BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4), #metode optimizer default
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.6899 - accuracy: 0.5538 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.6212 - accuracy: 0.6593 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.5638 - accuracy: 0.7046 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.5219 - accuracy: 0.7327 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.4455 - accuracy: 0.7994 - 34s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3406 - accuracy: 0.8649 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2705 - accuracy: 0.8980 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2274 - accuracy: 0.9158 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.2015 - accuracy: 0.9262 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1800 - accuracy: 0.9342 - 32s/epoch - 29ms/step
accuracy: 93.97%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.6630 - accuracy: 0.6039 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.5908 - accuracy: 0.6727 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 34s - loss: 0.5418 - accuracy: 0.7169 - 34s/epoch - 30ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4706 - accuracy: 0.7787 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3756 - accuracy: 0.8439 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3014 - accuracy: 0.8834 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2508 - accuracy: 0.9084 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2174 - accuracy: 0.9223 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1942 - accuracy: 0.9308 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1806 - accuracy: 0.9348 - 32s/epoch - 29ms/step
accuracy: 94.49%
Epoch 1/10


...

1122/1122 - 34s - loss: 0.6709 - accuracy: 0.5779 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 34s - loss: 0.5954 - accuracy: 0.6708 - 34s/epoch - 30ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.5254 - accuracy: 0.7317 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4293 - accuracy: 0.8070 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3463 - accuracy: 0.8575 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.2899 - accuracy: 0.8870 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2469 - accuracy: 0.9064 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2113 - accuracy: 0.9210 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1836 - accuracy: 0.9318 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 34s - loss: 0.1669 - accuracy: 0.9375 - 34s/epoch - 30ms/step
accuracy: 92.52%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.6997 - accuracy: 0.5573 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.5918 - accuracy: 0.6722 - 32s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.5351 - accuracy: 0.7195 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.4769 - accuracy: 0.7723 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 32s - loss: 0.3820 - accuracy: 0.8423 - 32s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 32s - loss: 0.3132 - accuracy: 0.8768 - 32s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.2652 - accuracy: 0.8988 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 34s - loss: 0.2255 - accuracy: 0.9159 - 34s/epoch - 30ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.1987 - accuracy: 0.9266 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.1801 - accuracy: 0.9334 - 32s/epoch - 29ms/step
accuracy: 93.16%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.7001 - accuracy: 0.5429 - 34s/epoch - 30ms/step
Epoch 2/10
1122/1122 - 32s - loss: 0.6473 - accuracy: 0.6132 - 32s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.6020 - accuracy: 0.6686 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 32s - loss: 0.5624 - accuracy: 0.7003 - 32s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.5179 - accuracy: 0.7380 - 34s/epoch - 31ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.4337 - accuracy: 0.8112 - 33s/epoch - 30ms/step
Epoch 7/10
1122/1122 - 32s - loss: 0.3230 - accuracy: 0.8783 - 32s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 32s - loss: 0.2622 - accuracy: 0.9038 - 32s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 32s - loss: 0.2248 - accuracy: 0.9178 - 32s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 32s - loss: 0.2006 - accuracy: 0.9267 - 32s/epoch - 29ms/step
accuracy: 91.49%
Model Accuracy List:  [93.97311210632324, 94.49021220207214, 92.52139925956726, 93.1562066078186, 91.48716330528259]
93.13% (+/- 1.06%)


In [ ]:
#RNN AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.SimpleRNN(100, return_sequences=True, activation='tanh'),
            layers.SimpleRNN(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


filename = "drive/MyDrive/Dann Dataset/RNN Model/rnn_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 36s - loss: 0.1879 - accuracy: 0.9173 - 36s/epoch - 32ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0607 - accuracy: 0.9769 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0375 - accuracy: 0.9857 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0326 - accuracy: 0.9870 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0579 - accuracy: 0.9814 - 33s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 35s - loss: 0.0320 - accuracy: 0.9877 - 35s/epoch - 31ms/step
Epoch 7/10
1122/1122 - 35s - loss: 0.0229 - accuracy: 0.9909 - 35s/epoch - 31ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0221 - accuracy: 0.9908 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0223 - accuracy: 0.9908 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0240 - accuracy: 0.9908 - 33s/epoch - 29ms/step
accuracy: 99.44%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.1944 - accuracy: 0.9121 - 34s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0534 - accuracy: 0.9799 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 34s - loss: 0.0375 - accuracy: 0.9854 - 34s/epoch - 30ms/step
Epoch 4/10
1122/1122 - 34s - loss: 0.0293 - accuracy: 0.9881 - 34s/epoch - 30ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0281 - accuracy: 0.9887 - 33s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0246 - accuracy: 0.9898 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0226 - accuracy: 0.9908 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0213 - accuracy: 0.9914 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0232 - accuracy: 0.9910 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0161 - accuracy: 0.9932 - 33s/epoch - 29ms/step
accuracy: 99.42%
Epoch 1/10


...

1122/1122 - 36s - loss: 0.1704 - accuracy: 0.9220 - 36s/epoch - 32ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0508 - accuracy: 0.9807 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 32s - loss: 0.0350 - accuracy: 0.9865 - 32s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0395 - accuracy: 0.9854 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0243 - accuracy: 0.9904 - 33s/epoch - 30ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0257 - accuracy: 0.9896 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0266 - accuracy: 0.9896 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 34s - loss: 0.0205 - accuracy: 0.9919 - 34s/epoch - 31ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0219 - accuracy: 0.9912 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0205 - accuracy: 0.9913 - 33s/epoch - 29ms/step
accuracy: 99.10%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.1727 - accuracy: 0.9257 - 34s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 33s - loss: 0.0643 - accuracy: 0.9763 - 33s/epoch - 29ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0462 - accuracy: 0.9822 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0328 - accuracy: 0.9870 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 34s - loss: 0.0301 - accuracy: 0.9879 - 34s/epoch - 31ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0273 - accuracy: 0.9890 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0247 - accuracy: 0.9900 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0242 - accuracy: 0.9901 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0194 - accuracy: 0.9923 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 33s - loss: 0.0179 - accuracy: 0.9928 - 33s/epoch - 29ms/step
accuracy: 99.24%
Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 34s - loss: 0.2104 - accuracy: 0.9060 - 34s/epoch - 31ms/step
Epoch 2/10
1122/1122 - 34s - loss: 0.0566 - accuracy: 0.9788 - 34s/epoch - 30ms/step
Epoch 3/10
1122/1122 - 33s - loss: 0.0379 - accuracy: 0.9856 - 33s/epoch - 29ms/step
Epoch 4/10
1122/1122 - 33s - loss: 0.0406 - accuracy: 0.9849 - 33s/epoch - 29ms/step
Epoch 5/10
1122/1122 - 33s - loss: 0.0305 - accuracy: 0.9877 - 33s/epoch - 29ms/step
Epoch 6/10
1122/1122 - 33s - loss: 0.0258 - accuracy: 0.9897 - 33s/epoch - 29ms/step
Epoch 7/10
1122/1122 - 33s - loss: 0.0202 - accuracy: 0.9920 - 33s/epoch - 29ms/step
Epoch 8/10
1122/1122 - 33s - loss: 0.0227 - accuracy: 0.9910 - 33s/epoch - 29ms/step
Epoch 9/10
1122/1122 - 33s - loss: 0.0199 - accuracy: 0.9914 - 33s/epoch - 29ms/step
Epoch 10/10
1122/1122 - 34s - loss: 0.0189 - accuracy: 0.9925 - 34s/epoch - 30ms/step
accuracy: 99.59%
Model Accuracy List:  [99.4365394115448, 99.41514134407043, 99.09771680831909, 99.23680424690247, 99.58987236022949]
99.36% (+/- 0.17%)


In [18]:
#GRU BEFORE TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10
1122/1122 - 158s - loss: 0.6897 - accuracy: 0.5373 - 158s/epoch - 141ms/step
Epoch 2/10
1122/1122 - 131s - loss: 0.6679 - accuracy: 0.5809 - 131s/epoch - 117ms/step
Epoch 3/10
1122/1122 - 117s - loss: 0.6613 - accuracy: 0.6095 - 117s/epoch - 104ms/step
Epoch 4/10
1122/1122 - 114s - loss: 0.6561 - accuracy: 0.6216 - 114s/epoch - 102ms/step
Epoch 5/10
1122/1122 - 116s - loss: 0.6515 - accuracy: 0.6264 - 116s/epoch - 103ms/step
Epoch 6/10
1122/1122 - 116s - loss: 0.6474 - accuracy: 0.6294 - 116s/epoch - 103ms/step
Epoch 7/10
1122/1122 - 114s - loss: 0.6435 - accuracy: 0.6315 - 114s/epoch - 102ms/step
Epoch 8/10
1122/1122 - 117s - loss: 0.6398 - accuracy: 0.6341 - 117s/epoch - 104ms/step
Epoch 9/10
1122/1122 - 114s - loss: 0.6362 - accuracy: 0.6367 - 114s/epoch - 102ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.6326 - accuracy: 0.6394 - 116s/epoch - 104ms/step
accuracy: 63.85%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 120s - loss: 0.6881 - accuracy: 0.5133 - 120s/epoch - 107ms/step
Epoch 2/10
1122/1122 - 114s - loss: 0.6755 - accuracy: 0.5149 - 114s/epoch - 102ms/step
Epoch 3/10
1122/1122 - 117s - loss: 0.6673 - accuracy: 0.5721 - 117s/epoch - 104ms/step
Epoch 4/10
1122/1122 - 115s - loss: 0.6607 - accuracy: 0.6107 - 115s/epoch - 103ms/step
Epoch 5/10
1122/1122 - 117s - loss: 0.6551 - accuracy: 0.6215 - 117s/epoch - 104ms/step
Epoch 6/10
1122/1122 - 114s - loss: 0.6502 - accuracy: 0.6273 - 114s/epoch - 102ms/step
Epoch 7/10
1122/1122 - 117s - loss: 0.6458 - accuracy: 0.6300 - 117s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 117s - loss: 0.6415 - accuracy: 0.6322 - 117s/epoch - 104ms/step
Epoch 9/10
1122/1122 - 114s - loss: 0.6375 - accuracy: 0.6347 - 114s/epoch - 102ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.6335 - accuracy: 0.6367 - 116s/epoch - 103ms/step
accuracy: 63.93%


Epoch 1/10


...

1122/1122 - 117s - loss: 0.6908 - accuracy: 0.5096 - 117s/epoch - 104ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.6762 - accuracy: 0.5405 - 116s/epoch - 104ms/step
Epoch 3/10
1122/1122 - 117s - loss: 0.6682 - accuracy: 0.5808 - 117s/epoch - 104ms/step
Epoch 4/10
1122/1122 - 115s - loss: 0.6615 - accuracy: 0.6097 - 115s/epoch - 102ms/step
Epoch 5/10
1122/1122 - 117s - loss: 0.6559 - accuracy: 0.6161 - 117s/epoch - 104ms/step
Epoch 6/10
1122/1122 - 115s - loss: 0.6510 - accuracy: 0.6215 - 115s/epoch - 102ms/step
Epoch 7/10
1122/1122 - 117s - loss: 0.6465 - accuracy: 0.6247 - 117s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 114s - loss: 0.6421 - accuracy: 0.6277 - 114s/epoch - 102ms/step
Epoch 9/10
1122/1122 - 116s - loss: 0.6380 - accuracy: 0.6308 - 116s/epoch - 103ms/step
Epoch 10/10
1122/1122 - 113s - loss: 0.6341 - accuracy: 0.6331 - 113s/epoch - 101ms/step
accuracy: 63.41%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 120s - loss: 0.6846 - accuracy: 0.5129 - 120s/epoch - 107ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.6721 - accuracy: 0.5434 - 116s/epoch - 104ms/step
Epoch 3/10
1122/1122 - 114s - loss: 0.6648 - accuracy: 0.5658 - 114s/epoch - 102ms/step
Epoch 4/10
1122/1122 - 116s - loss: 0.6591 - accuracy: 0.5809 - 116s/epoch - 104ms/step
Epoch 5/10
1122/1122 - 114s - loss: 0.6541 - accuracy: 0.5917 - 114s/epoch - 102ms/step
Epoch 6/10
1122/1122 - 116s - loss: 0.6496 - accuracy: 0.5992 - 116s/epoch - 104ms/step
Epoch 7/10
1122/1122 - 116s - loss: 0.6456 - accuracy: 0.6060 - 116s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 114s - loss: 0.6417 - accuracy: 0.6120 - 114s/epoch - 101ms/step
Epoch 9/10
1122/1122 - 116s - loss: 0.6381 - accuracy: 0.6175 - 116s/epoch - 103ms/step
Epoch 10/10
1122/1122 - 114s - loss: 0.6344 - accuracy: 0.6211 - 114s/epoch - 101ms/step
accuracy: 62.77%


Model Accuracy List:  [63.84579539299011, 63.92781734466553, 63.291728496551514, 63.40941786766052, 62.767475843429565]
63.45% (+/- 0.42%)


In [19]:
#GRU AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.GRU(100, return_sequences=True, activation='tanh'),
            layers.GRU(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/GRU Model/gru_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 119s - loss: 0.1998 - accuracy: 0.9084 - 119s/epoch - 106ms/step
Epoch 2/10
1122/1122 - 117s - loss: 0.0614 - accuracy: 0.9747 - 117s/epoch - 104ms/step
Epoch 3/10
1122/1122 - 114s - loss: 0.0339 - accuracy: 0.9871 - 114s/epoch - 102ms/step
Epoch 4/10
1122/1122 - 117s - loss: 0.1209 - accuracy: 0.9736 - 117s/epoch - 104ms/step
Epoch 5/10
1122/1122 - 117s - loss: 0.0238 - accuracy: 0.9911 - 117s/epoch - 104ms/step
Epoch 6/10
1122/1122 - 114s - loss: 0.0193 - accuracy: 0.9927 - 114s/epoch - 102ms/step
Epoch 7/10
1122/1122 - 117s - loss: 0.0146 - accuracy: 0.9947 - 117s/epoch - 104ms/step
Epoch 8/10
1122/1122 - 115s - loss: 0.0141 - accuracy: 0.9946 - 115s/epoch - 102ms/step
Epoch 9/10
1122/1122 - 117s - loss: 0.0139 - accuracy: 0.9946 - 117s/epoch - 104ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.0122 - accuracy: 0.9955 - 116s/epoch - 103ms/step
accuracy: 99.58%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

1122/1122 - 118s - loss: 0.2009 - accuracy: 0.9087 - 118s/epoch - 105ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.0553 - accuracy: 0.9775 - 116s/epoch - 103ms/step
Epoch 3/10
1122/1122 - 116s - loss: 0.0317 - accuracy: 0.9880 - 116s/epoch - 103ms/step
Epoch 4/10
1122/1122 - 114s - loss: 0.0223 - accuracy: 0.9914 - 114s/epoch - 102ms/step
Epoch 5/10
1122/1122 - 116s - loss: 0.6613 - accuracy: 0.8001 - 116s/epoch - 103ms/step
Epoch 6/10
1122/1122 - 113s - loss: 0.1165 - accuracy: 0.9584 - 113s/epoch - 101ms/step
Epoch 7/10
1122/1122 - 115s - loss: 0.0547 - accuracy: 0.9802 - 115s/epoch - 103ms/step
Epoch 8/10
1122/1122 - 115s - loss: 0.0394 - accuracy: 0.9854 - 115s/epoch - 103ms/step
Epoch 9/10
1122/1122 - 113s - loss: 0.0300 - accuracy: 0.9887 - 113s/epoch - 101ms/step
Epoch 10/10
1122/1122 - 116s - loss: 0.0250 - accuracy: 0.9903 - 116s/epoch - 103ms/step
accuracy: 98.98%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 119s - loss: 0.2170 - accuracy: 0.8975 - 119s/epoch - 106ms/step
Epoch 2/10
1122/1122 - 116s - loss: 0.0553 - accuracy: 0.9775 - 116s/epoch - 103ms/step
Epoch 3/10
1122/1122 - 113s - loss: 0.0344 - accuracy: 0.9871 - 113s/epoch - 100ms/step
Epoch 4/10
1122/1122 - 116s - loss: 0.0234 - accuracy: 0.9906 - 116s/epoch - 103ms/step
Epoch 5/10
1122/1122 - 115s - loss: 0.0185 - accuracy: 0.9926 - 115s/epoch - 102ms/step
Epoch 6/10
1122/1122 - 113s - loss: 0.0252 - accuracy: 0.9918 - 113s/epoch - 101ms/step
Epoch 7/10
1122/1122 - 115s - loss: 0.0152 - accuracy: 0.9938 - 115s/epoch - 102ms/step
Epoch 8/10
1122/1122 - 114s - loss: 0.0141 - accuracy: 0.9944 - 114s/epoch - 102ms/step
Epoch 9/10
1122/1122 - 113s - loss: 0.0137 - accuracy: 0.9945 - 113s/epoch - 101ms/step
Epoch 10/10
1122/1122 - 115s - loss: 0.0119 - accuracy: 0.9953 - 115s/epoch - 102ms/step
accuracy: 99.33%


Model Accuracy List:  [99.5756208896637, 99.33668375015259, 98.76961708068848, 98.98002743721008, 99.33309555053711]
99.20% (+/- 0.29%)


In [20]:
#LSTM BEFORE TUNING 

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.SGD(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_before_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

filename = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_before_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 126s - loss: 0.6782 - accuracy: 0.5242 - 126s/epoch - 112ms/step
Epoch 2/10
1122/1122 - 124s - loss: 0.6670 - accuracy: 0.6303 - 124s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 124s - loss: 0.6571 - accuracy: 0.6869 - 124s/epoch - 111ms/step
Epoch 4/10
1122/1122 - 125s - loss: 0.6471 - accuracy: 0.6965 - 125s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 125s - loss: 0.6366 - accuracy: 0.6991 - 125s/epoch - 111ms/step
Epoch 6/10
1122/1122 - 122s - loss: 0.6260 - accuracy: 0.7011 - 122s/epoch - 109ms/step
Epoch 7/10
1122/1122 - 125s - loss: 0.6154 - accuracy: 0.7016 - 125s/epoch - 112ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.6049 - accuracy: 0.7014 - 125s/epoch - 111ms/step
Epoch 9/10
1122/1122 - 125s - loss: 0.5950 - accuracy: 0.7005 - 125s/epoch - 111ms/step
Epoch 10/10
1122/1122 - 124s - loss: 0.5862 - accuracy: 0.6995 - 124s/epoch - 111ms/step
accuracy: 70.01%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

1122/1122 - 127s - loss: 0.6917 - accuracy: 0.5785 - 127s/epoch - 113ms/step
Epoch 2/10
1122/1122 - 124s - loss: 0.6768 - accuracy: 0.6708 - 124s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.6669 - accuracy: 0.6979 - 125s/epoch - 112ms/step
Epoch 4/10
1122/1122 - 123s - loss: 0.6571 - accuracy: 0.7016 - 123s/epoch - 110ms/step
Epoch 5/10
1122/1122 - 126s - loss: 0.6469 - accuracy: 0.7022 - 126s/epoch - 112ms/step
Epoch 6/10
1122/1122 - 125s - loss: 0.6360 - accuracy: 0.7031 - 125s/epoch - 112ms/step
Epoch 7/10
1122/1122 - 125s - loss: 0.6248 - accuracy: 0.7016 - 125s/epoch - 111ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.6135 - accuracy: 0.7012 - 125s/epoch - 111ms/step
Epoch 9/10
1122/1122 - 124s - loss: 0.6026 - accuracy: 0.6996 - 124s/epoch - 111ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.5927 - accuracy: 0.6977 - 125s/epoch - 111ms/step
accuracy: 69.58%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1122/1122 - 127s - loss: 0.6843 - accuracy: 0.5172 - 127s/epoch - 114ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.6688 - accuracy: 0.6362 - 125s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.6575 - accuracy: 0.6814 - 125s/epoch - 112ms/step
Epoch 4/10
1122/1122 - 126s - loss: 0.6467 - accuracy: 0.6913 - 126s/epoch - 112ms/step
Epoch 5/10
1122/1122 - 123s - loss: 0.6359 - accuracy: 0.6960 - 123s/epoch - 110ms/step
Epoch 6/10
1122/1122 - 124s - loss: 0.6246 - accuracy: 0.6989 - 124s/epoch - 111ms/step
Epoch 7/10
1122/1122 - 125s - loss: 0.6127 - accuracy: 0.7009 - 125s/epoch - 112ms/step
Epoch 8/10
1122/1122 - 126s - loss: 0.6017 - accuracy: 0.7015 - 126s/epoch - 112ms/step
Epoch 9/10
1122/1122 - 126s - loss: 0.5919 - accuracy: 0.7019 - 126s/epoch - 112ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.5834 - accuracy: 0.7010 - 125s/epoch - 111ms/step
accuracy: 70.45%


Model Accuracy List:  [70.0082004070282, 69.80849504470825, 69.86448168754578, 69.57560777664185, 70.44935822486877]
69.94% (+/- 0.29%)


In [21]:
#LSTM AFTER TUNING

fold_number = 0

kf = KFold(n_splits = total_fold, shuffle=True)                                            
cvscores = []

for train_index, test_index in kf.split(x_train):                                     
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]                             
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]  
    model = keras.Sequential(                                                   
        [
            keras.Input(shape=(None,5)),
            layers.LSTM(100, return_sequences=True, activation='tanh'),
            layers.LSTM(100, activation='tanh'),
            layers.Dense(2)
        ]
    )

    model.compile(
        loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer = keras.optimizers.Adam(lr=3e-4),
        metrics = ["accuracy"]
    )
    model.fit(x_train_fold, y_train_fold, batch_size=100, epochs=10, verbose=2)
    scores = model.evaluate(x_train[test_index], y_train[test_index], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    fold_number+=1
    fold_name = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_after_tuning_fold_" + str(fold_number) + ".sav"
    pickle.dump(model, open(fold_name, 'wb'))

print("Model Accuracy List: " , cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))


filename = "drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_after_tuning.sav"
pickle.dump(model, open(filename, 'wb'))


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 127s - loss: 0.2203 - accuracy: 0.9027 - 127s/epoch - 114ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.0972 - accuracy: 0.9645 - 125s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.0531 - accuracy: 0.9795 - 125s/epoch - 111ms/step
Epoch 4/10
1122/1122 - 125s - loss: 0.0409 - accuracy: 0.9837 - 125s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 125s - loss: 0.0298 - accuracy: 0.9881 - 125s/epoch - 111ms/step
Epoch 6/10
1122/1122 - 125s - loss: 0.0297 - accuracy: 0.9890 - 125s/epoch - 111ms/step
Epoch 7/10
1122/1122 - 124s - loss: 0.0232 - accuracy: 0.9906 - 124s/epoch - 111ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.0198 - accuracy: 0.9915 - 125s/epoch - 112ms/step
Epoch 9/10
1122/1122 - 125s - loss: 0.0179 - accuracy: 0.9928 - 125s/epoch - 112ms/step
Epoch 10/10
1122/1122 - 125s - loss: 0.0178 - accuracy: 0.9927 - 125s/epoch - 111ms/step
accuracy: 98.38%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

1122/1122 - 128s - loss: 0.2332 - accuracy: 0.8966 - 128s/epoch - 114ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.1086 - accuracy: 0.9583 - 125s/epoch - 111ms/step
Epoch 3/10
1122/1122 - 124s - loss: 0.0581 - accuracy: 0.9771 - 124s/epoch - 110ms/step
Epoch 4/10
1122/1122 - 124s - loss: 0.0612 - accuracy: 0.9797 - 124s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 125s - loss: 0.0337 - accuracy: 0.9872 - 125s/epoch - 111ms/step
Epoch 6/10
1122/1122 - 125s - loss: 0.0280 - accuracy: 0.9890 - 125s/epoch - 111ms/step
Epoch 7/10
1122/1122 - 124s - loss: 0.0249 - accuracy: 0.9902 - 124s/epoch - 111ms/step
Epoch 8/10
1122/1122 - 124s - loss: 0.0233 - accuracy: 0.9905 - 124s/epoch - 111ms/step
Epoch 9/10
1122/1122 - 125s - loss: 0.0243 - accuracy: 0.9902 - 125s/epoch - 111ms/step
Epoch 10/10
1122/1122 - 124s - loss: 0.0192 - accuracy: 0.9922 - 124s/epoch - 111ms/step
accuracy: 99.44%


Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


1122/1122 - 129s - loss: 0.2265 - accuracy: 0.9012 - 129s/epoch - 115ms/step
Epoch 2/10
1122/1122 - 125s - loss: 0.0805 - accuracy: 0.9678 - 125s/epoch - 112ms/step
Epoch 3/10
1122/1122 - 125s - loss: 0.0721 - accuracy: 0.9748 - 125s/epoch - 111ms/step
Epoch 4/10
1122/1122 - 125s - loss: 0.0442 - accuracy: 0.9833 - 125s/epoch - 111ms/step
Epoch 5/10
1122/1122 - 127s - loss: 0.0347 - accuracy: 0.9868 - 127s/epoch - 113ms/step
Epoch 6/10
1122/1122 - 127s - loss: 0.0276 - accuracy: 0.9893 - 127s/epoch - 113ms/step
Epoch 7/10
1122/1122 - 123s - loss: 0.0212 - accuracy: 0.9913 - 123s/epoch - 110ms/step
Epoch 8/10
1122/1122 - 125s - loss: 0.0243 - accuracy: 0.9905 - 125s/epoch - 112ms/step
Epoch 9/10
1122/1122 - 126s - loss: 0.0160 - accuracy: 0.9936 - 126s/epoch - 113ms/step
Epoch 10/10
1122/1122 - 126s - loss: 0.0187 - accuracy: 0.9928 - 126s/epoch - 112ms/step
accuracy: 99.08%


Model Accuracy List:  [98.38094115257263, 98.70190024375916, 98.84807467460632, 99.4436502456665, 99.08345341682434]
98.89% (+/- 0.36%)


Get list of accuracy, precision, recall and f1
Pengujian Data Testing
- Load Model untuk klasifikasi data Testing
- Model sudah di Load lalu melakukan Predict terhadap data Testing
- Simpan data Hasil Predict untuk codingan selanjutnya

In [22]:
#RNN K-FOLD BEFORE TUNING 
file_rnn_fold_before = [("drive/MyDrive/Dann Dataset/RNN Model/rnn_model_before_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_rnn_before = [(pickle.load(open(file_rnn_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_before = [(loaded_model_rnn_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_before = [(np.argmax(y_pred_rnn_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_before[ypvc] = np.reshape(y_pred_values_rnn_before[ypvc],(-1,1))
result_rnn_before = [(classification_report(y_test, y_pred_values_rnn_before[res], output_dict = True)) for res in range(total_fold)]

#RNN K-FOLD AFTER TUNING 
file_rnn_fold_after = [("drive/MyDrive/Dann Dataset/RNN Model/rnn_model_after_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_rnn_after = [(pickle.load(open(file_rnn_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_rnn_after = [(loaded_model_rnn_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_rnn_after = [(np.argmax(y_pred_rnn_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_rnn_after[ypvc] = np.reshape(y_pred_values_rnn_after[ypvc],(-1,1))
result_rnn_after = [(classification_report(y_test, y_pred_values_rnn_after[res], output_dict = True)) for res in range(total_fold)]

#GRU K-FOLD BEFORE TUNING
file_gru_fold_before = [("drive/MyDrive/Dann Dataset/GRU Model/gru_model_before_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_gru_before = [(pickle.load(open(file_gru_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_before = [(loaded_model_gru_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_before = [(np.argmax(y_pred_gru_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_before[ypvc] = np.reshape(y_pred_values_gru_before[ypvc],(-1,1))
result_gru_before = [(classification_report(y_test, y_pred_values_gru_before[res], output_dict = True)) for res in range(total_fold)]

#GRU K-FOLD AFTER TUNING
file_gru_fold_after = [("drive/MyDrive/Dann Dataset/GRU Model/gru_model_after_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_gru_after = [(pickle.load(open(file_gru_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_gru_after = [(loaded_model_gru_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_gru_after = [(np.argmax(y_pred_gru_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_gru_after[ypvc] = np.reshape(y_pred_values_gru_after[ypvc],(-1,1))
result_gru_after = [(classification_report(y_test, y_pred_values_gru_after[res], output_dict = True)) for res in range(total_fold)]

#LSTM K-FOLD BEFORE TUNING
file_lstm_fold_before = [("drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_before_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_lstm_before = [(pickle.load(open(file_lstm_fold_before[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_before = [(loaded_model_lstm_before[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_before = [(np.argmax(y_pred_lstm_before[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_before[ypvc] = np.reshape(y_pred_values_lstm_before[ypvc],(-1,1))
result_lstm_before = [(classification_report(y_test, y_pred_values_lstm_before[res], output_dict = True)) for res in range(total_fold)]

#LSTM K-FOLD AFTER TUNING
file_lstm_fold_after = [("drive/MyDrive/Dann Dataset/LSTM Model/lstm_model_after_tuning_fold_" + str(fn+1) + ".sav") for fn in range(total_fold)]
loaded_model_lstm_after = [(pickle.load(open(file_lstm_fold_after[lm], "rb"))) for lm in range(total_fold)]
y_pred_lstm_after = [(loaded_model_lstm_after[yp].predict(x_test)) for yp in range(total_fold)] 
y_pred_values_lstm_after = [(np.argmax(y_pred_lstm_after[ypv], axis = 1)) for ypv in range(total_fold)]
for ypvc in range(total_fold):
  y_pred_values_lstm_after[ypvc] = np.reshape(y_pred_values_lstm_after[ypvc],(-1,1))
result_lstm_after = [(classification_report(y_test, y_pred_values_lstm_after[res], output_dict = True)) for res in range(total_fold)]

1878/1878 [==============================] - 32s 17ms/step


Buat List untuk menampung dari Hasil dari Fold untuk Precision,Recall,Accuracy

In [23]:
precision_list_before = {}
precision_list_after = {}
recall_list_before = {}
recall_list_after = {}
accuracy_list_before = {}
accuracy_list_after = {}
f1_list_before = {}
f1_list_after = {}

Rata-rata dari Precision,Recall,F1-Score,Accuracy

Mengambil data hasil predict pada codingan sebelumnya


In [24]:
for sco in range(total_fold):
  precision_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['precision'] , result_gru_before[sco]['weighted avg']['precision'], result_lstm_before[sco]['weighted avg']['precision']]
  precision_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['precision'] , result_gru_after[sco]['weighted avg']['precision'], result_lstm_after[sco]['weighted avg']['precision']]
  recall_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['recall'] , result_gru_before[sco]['weighted avg']['recall'], result_lstm_before[sco]['weighted avg']['recall']]
  recall_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['recall'] , result_gru_after[sco]['weighted avg']['recall'], result_lstm_after[sco]['weighted avg']['recall']]
  f1_list_before[(sco+1)] = [result_rnn_before[sco]['weighted avg']['f1-score'] , result_gru_before[sco]['weighted avg']['f1-score'], result_lstm_before[sco]['weighted avg']['f1-score']]
  f1_list_after[(sco+1)] = [result_rnn_after[sco]['weighted avg']['f1-score'] , result_gru_after[sco]['weighted avg']['f1-score'], result_lstm_after[sco]['weighted avg']['f1-score']]
  accuracy_list_before[(sco+1)] = [result_rnn_before[sco]['accuracy'] , result_gru_before[sco]['accuracy'], result_lstm_before[sco]['accuracy']]
  accuracy_list_after[(sco+1)] = [result_rnn_after[sco]['accuracy'] , result_gru_after[sco]['accuracy'], result_lstm_after[sco]['accuracy']]

Menampilkan Hasil Predict Tiap Fold

In [25]:
print("{:<5} {:<0}".format("","Precision K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Precision K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in precision_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Recall K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Recall K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in recall_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","Accuracy K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","Accuracy K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in accuracy_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

print("{:<5} {:<0}".format("","F1-Score K-Fold Sebelum Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_before.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")
print("{:<5} {:<0}".format("","F1-Score K-Fold Sesudah Tuning"))
print ("{:<8} {:<15} {:<10} {:<10}".format('Fold','RNN','GRU','LSTM'))
for k, v in f1_list_after.items():
    rnn_fold_data, gru_fold_data, lstm_fold_data = v
    print ("{:<8} {:<15} {:<10} {:<10}".format(k, format(rnn_fold_data,".3f"), format(gru_fold_data,".3f"), format(lstm_fold_data,".3f") ))
print("")
print("")

      Precision K-Fold Sebelum Tuning
Fold     RNN             GRU        LSTM      
1        0.943           0.695      0.734     
2        0.944           0.728      0.747     
3        0.929           0.718      0.745     
4        0.938           0.710      0.731     
5        0.924           0.679      0.744     


      Precision K-Fold Sesudah Tuning
Fold     RNN             GRU        LSTM      
1        0.995           0.995      0.986     
2        0.995           0.994      0.988     
3        0.991           0.989      0.988     
4        0.993           0.989      0.995     
5        0.997           0.993      0.990     


      Recall K-Fold Sebelum Tuning
Fold     RNN             GRU        LSTM      
1        0.941           0.639      0.697     
2        0.943           0.636      0.699     
3        0.925           0.633      0.700     
4        0.932           0.635      0.696     
5        0.915           0.624      0.701     


      Recall K-Fold Sesudah Tuning
Fo